# Bayesian Logistic Regression (MCMC: Metropolis-Hastings)

This notebook uses **random-walk Metropolis-Hastings** to sample from the
posterior of logistic regression coefficients.

Because MCMC cost scales poorly with dataset size, we intentionally run on a
smaller universe and/or a subsample of observations.


# Chapter 10 Bayesian ML (Predictive): Bayesian Logistic Regression (MCMC: MH)

These notebooks mirror the *methods* highlighted in
`ml_finance_thoery/machine-learning-for-trading/10_bayesian_machine_learning/README.md`
and apply them to the local `dataset/cleaned/` asset universe to produce
out-of-sample predictions and a backtest using the same **vectorized** engine
used by the `notebooks/ML_Linear_Models_*` notebooks.


In [1]:
from __future__ import annotations

import math
from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'

# Ensure `src/` is on sys.path so `backtester` is importable
src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [2]:
from scipy.special import expit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

ASSET_LIMIT = 20
N_TRAIN_SAMPLES = 20_000
N_STEPS = 4_000
BURN = 1_000
STEP_SCALE = 0.05
TAU2 = 10.0**2

# Reuse the same feature builder as MAP notebook by importing it from that notebook is not possible.
# Keep a minimal feature set for speed: returns + range + volume z-score.

def compute_features_fast(ohlcv: pd.DataFrame) -> pd.DataFrame:
    df = ohlcv.copy().sort_index()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    ret_1d = c.pct_change()
    hl_range_pct = (h - l) / c.replace(0.0, np.nan)
    vol_z_20 = (v - v.rolling(20).mean()) / v.rolling(20).std()
    out = pd.DataFrame({'ret_1d': ret_1d, 'hl_range_pct': hl_range_pct, 'vol_z_20': vol_z_20}).sort_index()
    out['ret_1d_fwd'] = out['ret_1d'].shift(-1)
    out['y_up_fwd'] = (out['ret_1d_fwd'] > 0).astype(int)
    return out

from backtester.data import load_cleaned_assets
files = sorted([p for p in CLEANED_DIR.glob('Asset_*.csv')])
symbols = [p.stem for p in files][:ASSET_LIMIT]
assets_ohlcv = load_cleaned_assets(symbols=symbols, cleaned_dir=str(CLEANED_DIR))

frames = []
for sym, df in assets_ohlcv.items():
    feat = compute_features_fast(df)
    feat['Asset_ID'] = sym
    frames.append(feat)
data = pd.concat(frames, axis=0).sort_index().dropna(subset=['y_up_fwd'])

# Time split
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(data.index.unique()).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))

df_train = data.loc[(data.index >= train_start) & (data.index < val_start)].copy()
df_test = data.loc[(data.index >= test_start) & (data.index <= end)].copy()

feature_cols = [c for c in df_train.columns if c not in {'Asset_ID', 'ret_1d_fwd', 'y_up_fwd'}]
X_train = df_train[feature_cols].replace([np.inf, -np.inf], np.nan)
y_train = df_train['y_up_fwd'].astype(int).to_numpy()
X_test = df_test[feature_cols].replace([np.inf, -np.inf], np.nan)

imp = SimpleImputer(strategy='median')
scaler = StandardScaler()
Xtr = scaler.fit_transform(imp.fit_transform(X_train))
Xte = scaler.transform(imp.transform(X_test))
Xtr = np.hstack([np.ones((Xtr.shape[0], 1)), Xtr])
Xte = np.hstack([np.ones((Xte.shape[0], 1)), Xte])

# Subsample training rows for MCMC
if Xtr.shape[0] > N_TRAIN_SAMPLES:
    idx_s = rng.choice(Xtr.shape[0], size=N_TRAIN_SAMPLES, replace=False)
    Xtr_s = Xtr[idx_s]
    y_s = y_train[idx_s]
else:
    Xtr_s = Xtr
    y_s = y_train

def log_post(beta: np.ndarray) -> float:
    z = Xtr_s @ beta
    p = expit(z)
    eps = 1e-12
    ll = float(np.sum(y_s * np.log(p + eps) + (1 - y_s) * np.log(1 - p + eps)))
    lp = float(-0.5 * np.sum(beta[1:] ** 2) / TAU2)
    return ll + lp

beta = np.zeros(Xtr_s.shape[1])
lp = log_post(beta)
samples = []
accept = 0
for t in range(N_STEPS):
    prop = beta + rng.normal(0.0, STEP_SCALE, size=beta.shape)
    lp_prop = log_post(prop)
    if math.log(rng.random()) < (lp_prop - lp):
        beta, lp = prop, lp_prop
        accept += 1
    if t >= BURN:
        samples.append(beta.copy())
print('accept_rate:', accept / N_STEPS)
B = np.stack(samples, axis=0)

# Posterior predictive probabilities
p_samps = expit(Xte @ B.T)
p_mean = p_samps.mean(axis=1)

pred_long = pd.DataFrame({'Date': df_test.index, 'Asset_ID': df_test['Asset_ID'].to_numpy(), 'signal': p_mean - 0.5})
pred_matrix = pred_long.pivot_table(index='Date', columns='Asset_ID', values='signal', aggfunc='mean').sort_index()


accept_rate: 0.02975


In [3]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

# IMPORTANT: performance metrics should start when the model has signals.
# Keep the original prediction date range before reindexing to market data.
pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

# Align prediction matrix to available trade dates
pred_matrix = pred_matrix.reindex(close_prices.index)
# Restrict backtest to the prediction window (avoid 2016-start metrics).
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Bayes Logistic Regression (MH MCMC)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,39.909565,25.092596,1.371819,-18.137274
1,MPT,18.611542,12.051576,0.771102,-23.362502


,Bayes Logistic Regression (MH MCMC) - 1N
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1399095.648171
Equity Peak,1415358.920382
Total Return [%],39.909565
CAGR [%],25.092596
Volatility (ann) [%],17.372862
Sharpe,1.371819


,Bayes Logistic Regression (MH MCMC) - MPT
Start,2024-07-16 00:00:00
End,2026-01-16 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1186115.418255
Equity Peak,1192965.029119
Total Return [%],18.611542
CAGR [%],12.051576
Volatility (ann) [%],16.487491
Sharpe,0.771102
